In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from matplotlib import pyplot
import numpy as np
import pandas
from matplotlib import pyplot as plt

import sys
sys.path.append('../tasks')
from database_utils import Database

from data import ALL_REGRESSION_DATATSETS, ALL_CLASSIFICATION_DATATSETS

regression_datasets = list(ALL_REGRESSION_DATATSETS.keys())
regression_datasets.sort()

classification_datasets = list(ALL_CLASSIFICATION_DATATSETS.keys())
classification_datasets.sort()

# with Database('../results/results.db') as db:
#     db.delete('regression', {})


In [3]:
models_names = [['linear', 'lin'],
                ['variationally_sparse_gp', 'SVGP'],
                ['deep_gp_doubly_stochastic','DGP'],
                ['svm', 'svm'],
                ['knn', 'knn'],
                ['random_forest', 'rf'],
                ['gradient_boosting_machine', 'gbm'],
                ['adaboost', 'ab'],
                ['mlp', 'mlp'],
                ]

fields = ['dataset', 'N', 'D']
fields = fields + [m[1] for m in models_names]
results = {f:[] for f in fields}

for dataset in regression_datasets:
    results['dataset'].append(dataset)
    results['N'].append(ALL_REGRESSION_DATATSETS[dataset].N)
    results['D'].append(ALL_REGRESSION_DATATSETS[dataset].D)
    
    with Database('../results/results.db') as db:
        for model, name in models_names:
            res = db.read('regression', ['test_loglik'], {'model':model, 'dataset':dataset})
            
            if len(res) > 1:
                L = [float(l[0]) for l in res]
                m = np.average(L)
                if m <100 and m>-100:
                    stderr = np.std(L)/float(len(L))**0.5
                    r = '{:.3f} ({:.3f})'.format(m, stderr)
                else:
                    r = 'nan'
            else:
                r = ''
            results[name].append(r)

# print(pandas.DataFrame(results).to_latex(columns=fields, index=False))
print(pandas.DataFrame(results, columns=fields))

     dataset      N   D             lin            SVGP            DGP  \
0     boston    506  13  -0.644 (0.021)  -0.189 (0.019)                  
1   concrete   1030   8  -0.953 (0.016)  -0.353 (0.017)                  
2     energy    768   8  -0.220 (0.036)   1.241 (0.013)                  
3     kin8nm   8192   8  -1.151 (0.006)  -0.383 (0.005)                  
4      naval  11934  12  -0.489 (0.005)   0.726 (0.460)                  
5      power   9568   4  -0.098 (0.010)   0.031 (0.011)  0.031 (0.014)   
6    protein  45730   9  -1.257 (0.002)  -1.110 (0.002)                  
7    winered   1599  11  -1.208 (0.019)  -1.163 (0.021)                  
8  winewhite   4898  12  -1.254 (0.012)  -1.167 (0.009)                  
9      yacht    308   6  -0.929 (0.026)   1.956 (0.043)  2.273 (0.061)   

              svm             knn              rf             gbm  \
0  -0.157 (0.026)  -0.467 (0.042)  -0.940 (0.087)  -0.642 (0.077)   
1  -0.558 (0.034)  -0.828 (0.032)  -1.871 (0.18

In [4]:
fields = ['dataset', 'N', 'D', 'K']

models_names = [['linear', 'lin'],
                ['variationally_sparse_gp', 'SVGP'],
                ['deep_gp_doubly_stochastic','DGP'],
                ['svm', 'svm'],
                ['knn', 'knn'],
                ['naive_bayes', 'nb'],
                ['decision_tree', 'dt'],
                ['random_forest', 'rf'],
                ['gradient_boosting_machine', 'gbm'],
                ['adaboost', 'ab'],
                ['mlp', 'mlp'],
                ]
                
fields = fields + [m[1] for m in models_names]
results = {f:[] for f in fields}

with Database('../results/results.db') as db:

    for dataset in classification_datasets:
        results['dataset'].append(dataset[:10])
        results['N'].append(ALL_CLASSIFICATION_DATATSETS[dataset].N)
        results['D'].append(ALL_CLASSIFICATION_DATATSETS[dataset].D)
        results['K'].append(ALL_CLASSIFICATION_DATATSETS[dataset].K)

        for model, name in models_names:
            res = db.read('classification', ['test_loglik'], {'model':model, 'dataset':dataset})
            if len(res)>0:
                L = [float(l[0]) for l in res]

                m = np.average(L)
                r = '{:.3f}'.format(m)  # only one split run
    #             stderr = np.std(L)/float(len(L))**0.5
    #             r = '{:.3f} ({:.3f})'.format(m, stderr)
            else:
                r = ''
            results[name].append(r)

print(pandas.DataFrame(results, columns=fields))#.to_latex(index=False))

#NB the DGP results here are with a too small learning rate but it will do for now. 

        dataset      N    D   K     lin    SVGP     DGP     svm     knn  \
0       abalone   4177    9   3  -0.754  -2.214  -2.196  -0.730  -2.169   
1    acute-infl    120    7   2  -0.069  -0.009  -0.317  -0.019  -0.000   
2    acute-neph    120    7   2  -0.032  -0.006  -0.315  -0.020  -0.000   
3         adult  48842   15   2  -0.348  -0.323  -0.322  -0.363  -1.109   
4     annealing    898   32   5  -0.479  -1.645  -1.420  -0.509  -1.197   
5    arrhythmia    452  263  13  -1.369  -1.390  -1.266  -1.213  -7.528   
6    audiology-    196   60  18  -1.181  -1.072  -1.197  -1.167  -2.101   
7    balance-sc    625    5   3  -0.178  -0.036  -0.053  -0.176  -0.652   
8      balloons     16    5   2  -1.341  -1.051  -0.933  -0.885  -0.714   
9          bank   4521   17   2  -0.259  -0.248  -0.249  -0.277  -1.287   
10        blood    748    5   2  -0.510  -0.509  -0.494  -0.539  -1.854   
11   breast-can    286   10   2  -0.182  -0.121  -0.196  -0.176  -0.406   
12   breast-can    699   

In [5]:

# fields = ['dataset', 'N', 'D', 'K']

# models_names = [['linear', 'lin'],
#                 ['variationally_sparse_gp', 'SVGP'],
#                 ['deep_gp_doubly_stochastic','DGP'],
#                 ['svm', 'svm'],
#                 ['knn', 'knn'],
#                 ['naive_bayes', 'nb'],
#                 ['decision_tree', 'dt'],
#                 ['random_forest', 'rf'],
#                 ['gradient_boosting_machine', 'gbm'],
#                 ['adaboost', 'ab'],
#                 ['mlp', 'mlp'],
#                 ]
                
# colours = ['C{}'.format(i) for i in range(10)]

# fields = fields + [m[1] for m in models_names]
# results = {f:[] for f in fields}


# for dataset in classification_datasets[:10]:  # don't show them all...
    
#     fig, axs = plt.subplots(1, 2, figsize=(10, 5))

#     results['dataset'].append(dataset)
#     results['N'].append(ALL_CLASSIFICATION_DATATSETS[dataset].N)
#     results['D'].append(ALL_CLASSIFICATION_DATATSETS[dataset].D)
#     results['K'].append(ALL_CLASSIFICATION_DATATSETS[dataset].K)

#     for (name, model), c in zip(models_names, colours):
#         with Database('../results/results.db') as db:
#             d = {'iterations':50, 'model':model, 'dataset':dataset}

#             res = db.read('active_learning', ['test_loglik', 'total_acc'], d) 
#         if len(res)>0:
#             test_ll = res[0][0]
#             test_acc = res[0][1]

#             axs[0].plot(test_ll, label=model, color=c)
#             axs[1].plot(test_acc, label=model, color=c)

#     plt.title('{} {} {} {}'.format(dataset,
#                                    ALL_CLASSIFICATION_DATATSETS[dataset].N,
#                                    ALL_CLASSIFICATION_DATATSETS[dataset].D,
#                                    ALL_CLASSIFICATION_DATATSETS[dataset].K))
#     plt.legend()
#     plt.show()
